# Compute escape scores

## Import Python modules and initialize directories

In [1]:
import os
import sys
import glob
import pandas as pd
from collections import defaultdict
import subprocess
import multiprocessing

resultsdir = '../results/predicted_escape_scores/'
if not os.path.isdir(resultsdir):
    os.makedirs(resultsdir)

## Compile input DMS data

Read in processed data from `process_data.ipynb`.

In [2]:
# Read in site-level escape scores
escape_calc_resultsdir = '../results/processed_input_data/'
site_escape_df = pd.read_csv(os.path.join(escape_calc_resultsdir, 'escape.csv'))

# Read in data on antibody sources
ab_source_df = pd.read_csv(os.path.join(escape_calc_resultsdir, 'antibody_sources.csv'))

assert(site_escape_df['antibody'].nunique() == len(ab_source_df))
print(f'Read in data for {len(ab_source_df)} antibodies')

Read in data for 1603 antibodies


For each of the above antibodies, make a dataframe with site-specific amino-acid escape value, where each entry at a given site corresponds to the site's value from `site_escape_df`. This strategy assumes that all mutations at a site have the same effect. The Bloom lab's escape calculator also makes this assumption, the reason being the amino-acid level measurements are noisy and site-level measurements are less so. These dataframes will serve as input for `dmsa-pred` below.

In [3]:
# Read in a complete list of sites and wildtype aminio acids for the RBD
site_df = pd.read_csv('../data/RBD_sites.csv')
site_df.rename(columns={
    'site_SARS2' : 'site',
    'amino_acid_SARS2' : 'wildtype'
}, inplace=True)

# Make dataframe with amino-acid-level escape scores
aa_escape_dict = defaultdict(list)
sites = list(set(site_escape_df['site']))
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
for (i, row) in site_df.iterrows():
    site = int(row.site)
    wt = str(row.wildtype)
    for mut in amino_acids:
        if wt == mut:
            continue
        aa_escape_dict['site'].append(site)
        aa_escape_dict['aa_sub'].append(f'{wt}{site}{mut}')

aa_escape_df = pd.DataFrame(aa_escape_dict)
aa_escape_df = aa_escape_df.merge(site_escape_df, on='site')

# For each antibody, write a dataframe with amino-acid-level escape scores
cao_dms_fs = []
for (antibody, data) in aa_escape_df.groupby('antibody'):
    output_f = os.path.join(resultsdir, f'{antibody}_mut_effects.csv')
    cao_dms_fs.append(output_f)
    if not os.path.isfile(output_f):
        data.to_csv(output_f, index=False)

## Compute escape scores from DMS data

Make a list of input files from Bloom lab DMSs.

In [4]:
bloom_dms_fs = [
    '../ncov-dmsa/my_profiles/escape-frac-data/Dong_2021_AZD1061.csv',
    '../ncov-dmsa/my_profiles/escape-frac-data/Dong_2021_AZD8895.csv',
    '../ncov-dmsa/my_profiles/escape-frac-data/Starr_2021_LY-CoV555.csv',
    '../ncov-dmsa/my_profiles/escape-frac-data/Starr_2021_LY-CoV016.csv',
    '../ncov-dmsa/my_profiles/escape-frac-data/Starr_2021_REGN10933.csv',
    '../ncov-dmsa/my_profiles/escape-frac-data/Starr_2021_REGN10987.csv'
 ]

For each antibody from both the Cao and Bloom datasets, use `dmsa-phenotype` to compute an escape score for each virus in an input multiple-sequence alignment.

In [5]:
# Define input params related to alignment
alignment = '../ncov-dmsa/results/global-build/translations/aligned.gene.S_withInternalNodes.fasta'
dms_wt_seq_id = 'Wuhan-Hu-1/2019'

# Get DMS inputs
cao_dms_inputs = [(f, 'normalized_escape', 'aa_sub') for f in cao_dms_fs]
bloom_dms_inputs = [(f, 'mut_escape', 'aa_mutation') for f in bloom_dms_fs]
dms_inputs = cao_dms_inputs + bloom_dms_inputs

# Cycle over each antibody with DMS data and compute escape scores
commands = []
for (i, (mut_effects_f, mut_effect_col, mutation_col)) in enumerate(dms_inputs, 1):

    # Define output files
    ab_label = os.path.basename(mut_effects_f).replace('_mut_effects.csv', '').replace('.csv', '')
    output_json = os.path.join(resultsdir, f'{ab_label}_escape_pred.json')
    output_df = os.path.join(resultsdir, f'{ab_label}_escape_pred.csv')
    
    # Run dmsa-pred
    cmd = ' '.join([
        'python',
        '../ncov-dmsa/my_profiles/dmsa-pred/dmsa_pred.py phenotype-prediction',
        '--model-type additive',
        f'--alignment {alignment}',
        f'--dms-wt-seq-id {dms_wt_seq_id}',
        f'--mut-effects-df {mut_effects_f}',
        f'--mut-effect-col {mut_effect_col}',
        f'--mutation-col {mutation_col}',
        '--mask-seqs-with-disallowed-aa-subs False',
        f'--experiment-label {ab_label}',
        f'--output-json {output_json}',
        f'--output-df {output_df}'
    ])
    if not os.path.isfile(output_df):
        commands.append(cmd)
        #print(i, ab_label)
        #! {cmd}

Run the above list of commands in parallel

In [6]:
def run_command(command):
    """Runs a given command in a subprocess."""
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    return stdout.decode(), stderr.decode()

with multiprocessing.Pool() as pool:
    results = pool.map(run_command, commands)

for stdout, stderr in results:
    #print("Output:", stdout)
    if stderr:
        print("Error:", stderr)

## Read in escape scores and merge with Nextstrain metadata

In [7]:
# Read in predicted escape scores
fs = glob.glob(os.path.join(resultsdir, '*_escape_pred.csv'))
dfs = []
for f in fs:
    antibody = os.path.basename(f).replace('_escape_pred.csv', '')
    df = pd.read_csv(f)
    df = df[~df['strain'].str.contains('NODE')]
    df['antibody'] = antibody
    dfs.append(df)
pred_df = pd.concat(dfs)
pred_df.rename(columns={'pred_phenotype':'escape_score'}, inplace=True)

# Read in metadata from Nextstrain
metadata_f = '../ncov-dmsa/results/global-build/global-build_subsampled_metadata.tsv.xz'
metadata_df = pd.read_csv(
    metadata_f,
    compression='xz', sep='\t', on_bad_lines='skip'
)
metadata_df['date'] = pd.to_datetime(metadata_df['date'])
metadata_df['year'] = metadata_df['date'].apply(
    lambda x: x.year + ((x.dayofyear - 1) / 365)
)
metadata_df['time'] = metadata_df['year'] - metadata_df['year'].min()

# Merge with prediction dataframe
pred_df = pred_df.merge(
    metadata_df[['strain', 'date', 'year', 'time', 'Nextstrain_clade']],
    on='strain', how='left'
)

# Report stats and save data to file
n_abs = len(pred_df['antibody'].unique())
n_strains = len(pred_df['strain'].unique())
assert sum(pred_df['strain'].value_counts() != n_abs) == 0
print('Read in data for')
print(n_abs, 'antibodies')
print(n_strains, 'strains')

output_f = os.path.join(resultsdir, 'all_predictions.csv')
if not os.path.isfile(output_f):
    print('Saving data to', output_f)
    pred_df.to_csv(output_f, index=False)

Read in data for
1609 antibodies
3942 strains
Saving data to ../results/predicted_escape_scores/all_predictions.csv
